In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras import backend as K

import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
import time

In [18]:
x_train = []
#x_test = []
y_train = []

df_train = pd.read_csv('train.csv')
#df_test = pd.read_csv('../input/sample_submission.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values[:40479], miniters=1000):
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    #x_train.append(img)    
    x_train.append(cv2.resize(img, (139, 139)))#139 minimum size for inception
    y_train.append(targets)

#for f, tags in tqdm(df_test.values, miniters=1000):
#    img = cv2.imread('../input/test-jpg/{}.jpg'.format(f))
#    x_test.append(cv2.resize(img, (32, 32)))
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32) / 255.
#x_test  = np.array(x_test, np.float32) / 255.

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [01:02<00:00, 646.04it/s]

(40479, 139, 139, 3)
(40479, 17)


In [19]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(36431, 139, 139, 3)
(36431, 17)
(4048, 139, 139, 3)
(4048, 17)


In [21]:
base_model = InceptionV3(input_shape=(139,139,3), weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
#fully connected layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(17, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

In [22]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=[])
#callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]
model.fit(x = x_train, y= y_train, validation_data=(x_val, y_val),
                  batch_size=128,verbose=2, epochs=3,#callbacks=callbacks,
                  shuffle=True)

y_pred = model.predict(x_val, batch_size = 128, verbose=2)
print("F2 score:",fbeta_score(y_val, np.array(y_pred)>0.2, beta=2, average='samples'))

Train on 36431 samples, validate on 4048 samples
Epoch 1/3
129s - loss: 0.1810 - val_loss: 0.1594
Epoch 2/3
124s - loss: 0.1390 - val_loss: 0.1352
Epoch 3/3
124s - loss: 0.1316 - val_loss: 0.1401
F2 score: 0.876142811477


In [23]:
def multilabelmetrics(y_true,y_pred):
    '''y_true and y_pred should be boolean np arrays
    of shape num_example x num_classes '''
    total = np.sum(y_true,axis = 0)
    tp = np.sum(y_true*y_pred,axis=0)
    tn = np.sum((1-y_true)*(1-y_pred),axis=0)
    fp = np.sum((1-y_true)*y_pred,axis=0)
    fn = np.sum(y_true*(1-y_pred),axis=0)
    return total,tp,tn,fp,fn

In [16]:
total,tp,tn,fp,fn = multilabelmetrics(y_val,np.array(y_pred)>0.2)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
water,225,2951,3711,1209,1434
habitation,476,449,6923,248,724
slash_burn,35,56,8005,0,35
artisinal_mine,66,157,7873,0,66
bare_ground,160,140,7792,4,164
blooming,79,86,7931,0,79
selective_logging,57,149,7889,1,58
road,502,1187,5308,1099,1601
blow_down,15,218,7863,0,15
agriculture,308,2717,2965,2106,2414
